## stock csv reader

In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import os

class stock_csv_read:
    def __init__(self, data, x_frames, y_frames):
        self.data = data
        self.x_frames = x_frames
        self.y_frames = y_frames
        self.stock_data = self.data_loader()
        
    def data_loader(self):
        stock_data = pd.read_csv("./data/kdd17/ourpped/"+self.data,header=None)
        stock_data = stock_data[29:]
        stock_data = stock_data.iloc[:,:12]
        target = pd.DataFrame(np.where(stock_data.iloc[:,11]>=0,1,0))
        stock_data = stock_data.iloc[:,:11].reset_index(drop=True)
        stock_data = pd.concat([stock_data,target],axis=1 , ignore_index= True)
        
        return stock_data
    
    def spliter(self, data):
        self.dd = data
        data_list = []
        for i in range(len(self.dd)-self.x_frames-self.y_frames+1):
            xy = []
            X = self.dd.iloc[i : i +self.x_frames, 0:11]
            y = self.dd.iloc[i +self.x_frames : i +self.x_frames +self.y_frames, 11:]
            xy.append(X)
            xy.append(y)
            data_list.append(xy)
        return data_list
    
    def cv_split(self):
        stock_data = self.stock_data
        
        data_len = len(stock_data)
        mok = data_len//19
        
        adder = 0
        data_list = []
        for i in range(10):
            sp_data = stock_data.iloc[0+adder:10*mok+adder,:]
            
            tvt = []
            train_sp_data = sp_data[0:8*mok]
            validation_sp_data = sp_data[8*mok:9*mok]
            test_sp_data = sp_data[9*mok:10*mok]
            
            train_sp_data_ = self.spliter(train_sp_data)
            validation_sp_data_ = self.spliter(validation_sp_data)
            test_sp_data_ = self.spliter(test_sp_data)

            tvt.append(train_sp_data_)
            tvt.append(validation_sp_data_)
            tvt.append(test_sp_data_)
            adder += mok
            data_list.append(tvt)
            
        return data_list
